In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
import logging
import tempfile
import pandas as pd
import os
import wandb
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# Login to Weights & Biases
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


##1.3 Data segregation


In [ ]:
# initiate wandb project
run = wandb.init(entity="flateam", project="the_simpsons_characters", job_type="split_data")

img_path = run.use_artifact('flateam/the_simpsons_characters/img_data:v1', type='raw_data').file()
df_img = np.load(img_path)

labels_path = run.use_artifact('flateam/the_simpsons_characters/label_data:v1', type='raw_data').file()
df_labels = pd.read_csv(labels_path)
df_labels = df_labels['character name']
df_labels = df_labels.values

In [ ]:
data_img_train, data_img_test, labels_train, labels_test = train_test_split(df_img, df_labels, test_size=0.2, random_state=42, stratify=df_labels)

## Saving the date in a npy archive

In [ ]:
np.save('img_data_train.npy', data_img_train)
loaded_data_train = np.load('img_data_train.npy')

np.save('img_data_test.npy', data_img_test)
loaded_data_test = np.load('img_data_test.npy')

## Saving the labels in a csv

In [ ]:
data = {"character name" : labels_train}
df = pd.DataFrame(data)
df.to_csv("label_data_train.csv", index=False)

data = {"character name" : labels_test}
df = pd.DataFrame(data)
df.to_csv("label_data_test.csv", index=False)

##Send to wandb

In [ ]:
#creating a wandb artifact
artifact = wandb.Artifact(name="img_data_train",
                                  type="split_data",
                                  description="train data of the images")

# add the file to the artifact
artifact.add_file('img_data_train.npy')
# save the artifact in W&B
wandb.run.log_artifact(artifact)

#creating a wandb artifact
artifact2 = wandb.Artifact(name="img_data_test",
                                  type="split_data",
                                  description="test data of the images")

# add the file to the artifact
artifact2.add_file('img_data_test.npy')
# save the artifact in W&B
wandb.run.log_artifact(artifact2)

#creating a wandb artifact
artifact3 = wandb.Artifact(name="label_data_train",
                                  type="split_data",
                                  description="labels of the train imagens")

# add the file to the artifact
artifact3.add_file('label_data_train.csv')
# save the artifact in W&B
wandb.run.log_artifact(artifact3)

#creating a wandb artifact
artifact4 = wandb.Artifact(name="label_data_test",
                                  type="split_data",
                                  description="labels of the test imagens")

# add the file to the artifact
artifact4.add_file('label_data_test.csv')
# save the artifact in W&B
wandb.run.log_artifact(artifact4)

<Artifact label_data_test>

In [ ]:
# closes wandb run
run.finish()